# Set kernel to: `py-deepLearning`
### NOT: `py-deeplearning` **OR** `py-pythonEnv`
If this kernel is not available in the `change kernel` options, please follow the Kernel Installation Instructions in the appendix found at the end of the notebook, before proceeding to the exercises below.  

# Data Science on the C3 AI Suite
## C3DS - Deep Learning
#### For questions, comments, or concerns, please email the C3.ai Academy Team \< <training@c3.ai> \>, discuss with your Trainer, or leave your thoughts in the Google Forms at the end of this module.

## Exercise Goal

<b>Objective: </b>This notebook is about **how to perform deep learning on the platform**. We will build a Deep Learning model in three different ways, train it with data, score model performance, and finally use it to make predictions on the platform. 
    
<b>Use Case: </b>The deep learning model that we are going to build is for a Lightbulb predictive maintenance use-case, i.e., we will continue to use lightbulb data as we have been doing in this course, and predict failures on the same using our newly trained model.  

- In Section 1, we produce the training and testing data set. Along the process, we review various API calls that we use to **get data and manipulate data on the platform**. This section should be familiar to you, as you have seen most of the commands earlier in the course.


- In Section 2, we **get familiar with Tensorflow**, specifically with the concepts of Tensors and computational graphs. We build a deep learning model using Tensorflow's low-level api, and run both forward and backward propagation. The goal of this section is to help you understand Tensorflow better and get comfortable with the concept of a computational graph, which is why we use the low-level api to build the model. 

    
- In Section 3, we upsert the serialized deep learning model for training on the platform. We initialize the model here using Tensorflow's high level api that abstracts all of the computational graph construction. It makes model initialization simple like sklearn.  We demonstrate **how to Train, Score, and do Prediction using your DNN on the platform**. Along the process, we **get familiar with various c3.MLPipe API calls**.


- In Section 4, we show another way to doing deep learning on the platform, by constructing a Machine Learning pipeline with both scaling and deep learning estimator as two separate steps, but part of the same pipeline. We then demonstrate **how to Train, Score and do Predictions using the pipeline on the platform**.



## Table of Contents

* [Section 1 - Extract the Data \& PreProcess](#1)

   * [A. Produce the Dataset](#1a)
   * [B. Clean \& Train-Test split the Dataset](#1b)
   * [C. Normalize the Features](#1c)
   * [D. Eyeball the Data](#1d)
   

* [Section 2 - Construct a Deep Neural Network using Tensorflow's Low-Level API](#2) 

   * [A. Construct the Computational Graph](#2a)
   * [B. Run Forward Propagation](#2b)
   * [C. Run Backward Propagation (Optional)](#2c)
   * [D. Assignment: Construct Your Own Computational Graph \& Run Forward Propagation](2#d)


* [Section 3 - Train the Deep Neural Network (DNN) on the Platform](#4)
   
   * [A. Construct a DNN Classifier using Tensorflow's High-Level API](#3a)
   * [B. Train on the Platform](#3b)
   * [C. Upsert \& Fetch the Model](#3c)
   * [D. Cleanup](#3d)
   * [E.. Assignment: Construct Your Own C3 Tensorflow Classifier \& Train on the Platform](#3e)
   

* [Section 4 - Train the ML Pipeline with DNN on the Platform](#4)

   * [A. Construct an ML Pipeline that Consists of a DNN Classifier](#4a)
   * [B. Train on the Platform](#4b)
   * [C. Upsert and fetch](#4c)
   * [D. Cleanup](#4d)
   * [E. Assignment: Construct your own ML Pipeline with DNN \& Train on the Platform](#4e)
   
[Appendix - Kernel Installation Instructions](#apdx)

**Before** we get started, lets import the required packages and define some helper functions we will use later in this exercise. The helper functions `serialize_compress_python_object` and `decompress_deserialize_python_object` are seentially used server-side to serialize and deserialize python objects, while functions `plot_roc` and `plot_prec` are used to plot the ROC and precision-recall curves respectively. You can reference matplotlib documentation [here](https://matplotlib.org/3.1.1/contents.html) if you are interested in understanding the code in the functions for plotting.

#### Import Packages:

In [ ]:
# Future proofing code below to be Python 3 compatible
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sklearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


import pandas as pd
import tensorflow as tf
from datetime import datetime
import numpy as np
import jsonpickle as jpickle
plt.rcParams['figure.figsize'] = (15,8)
import sys
sys.path.append('.')

#### Define Helper Functions (e.g., plot ROC)

In [ ]:
# HELPER FUNCTIONS

# The code below is used by server-side python to serialized/deserialize python objects
# In the future, this function will be made available to the user (via pyc3 / separate package / python client side action / ...)
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
import numpy as np


def serialize_compress_python_object(obj):
    """Serialize and compress a python object."""
    import base64
    import dill
    import zlib
    return base64.b64encode(zlib.compress(dill.dumps(obj, protocol=2))).decode('utf-8')

def decompress_deserialize_python_object(clob):
    """Decompress and deserialize a clob to a python object."""
    import base64
    import dill
    import zlib
    return dill.loads(zlib.decompress(base64.b64decode(clob.encode('utf-8'))))



def plot_roc(y_pred_test, y_true_test, y_pred_train, y_true_train, fp=None, save=False):
    
    fpr, tpr, thresholds = roc_curve(y_true=y_true_test, y_score=y_pred_test)
    roc_auc_test = auc(fpr, tpr)
    lw = 1
    plt.figure(figsize=[6, 6])
    plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f), Testing' % np.round(roc_auc_test, 3))
    
    fpr, tpr, thresholds = roc_curve(y_true=y_true_train, y_score=y_pred_train)
    roc_auc_train = auc(fpr, tpr)
    lw = 1
    plt.plot(fpr, tpr, color='blue',
         lw=lw, label='ROC curve (area = %0.2f), Training' % np.round(roc_auc_train, 3))
    
    
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    if save:
        plt.savefig(fp)
    plt.show()
    return roc_auc_test

def plot_prec(y_pred_test, y_true_test, y_pred_train, y_true_train, fp=None, save=False):
    
    prec, rec, thresholds = precision_recall_curve(y_true=y_true_test, probas_pred=y_pred_test)
    avg_prec_test = average_precision_score(y_true=y_true_test, y_score=y_pred_test) 
    lw = 1
    plt.figure(figsize=[6, 6])
    plt.plot(rec, prec, color='darkorange',
         lw=lw, label='Precision-Recall curve(average precision = %0.2f), Testing' % avg_prec_test)
    
    prec, rec, thresholds = precision_recall_curve(y_true=y_true_train, probas_pred=y_pred_train)
    avg_prec_train = average_precision_score(y_true=y_true_train, y_score=y_pred_train) 
    lw = 1
    
    plt.plot(rec, prec, color='blue',
         lw=lw, label='Precision-Recall curve(average precision = %0.2f), Training' % avg_prec_train)
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall curve')
    plt.legend(loc="lower right")
    if save:
        plt.savefig(fp)
    plt.show()
    return avg_prec_test

# Section 1 - Extract Labeled Data \& Setup <a class="anchor" id="1"></a>

In Section 1, we will extract the labelled data and set it up for deep learning. We will create failure label metrics, clean and split the data for training and testing, and normalize the features we want to use. You are probably familiar now with most of the commands seen in this section. 

We go through the following steps:

* **Produce the Dataset**: First, we'll create the failure label metrics, then construct a `EvalMetricsspec`, at the end, we'll issue the `EvalMetricswithMetadata` API call to get the data. We are going to predict the probability of the lightbulb failing in the next **120 days**. The features that we use are `'DurationOnInHours', 'StandardDeviationWattsPreviousWeek', and 'SwitchCountPreviousWeek'`.
    
* **Clean \& Train-Test Split the Dataset**: Using 2014 data, randomly split into train and test. We use only those datapoints where the lightbulb has never failed before.

* **Normalize the Features**: We use C3 machine learning pipeline to normalize the features on the platform, so that the training process is optimized.

### A. Produce the Dataset <a class="anchor" id="1a"></a>

Below, produce the dataset by fetching the IDs of the lightbulbs. Then, use a loop or list comprehension to extract those `id`'s to a list:

In [ ]:
#fetch only the IDs of the lightbulbs by creating a FetchSpec type
fetch_spec = c3.FetchSpec(include='id')
#pass the fetch_spec while fetching data from SmartBulb
fetch_res = c3.SmartBulb.fetch(spec=fetch_spec)

smb_ids = [] 
for i in fetch_res.objs:
    smb_ids.append(i['id'])

#view the fetched ids
smb_ids

Next, we create the failure label metric as a compund metric using the `window` expression engine function which you have encountered in metrics module of the C3 Fundamentals Course. We also include `HasEverFailed` as an expression for the EvalMetricsSpec to keep only the datapoints for which the lightbulb has never failed before.

(Optional) It might be a good idea to use eval to force evaluation with "DAY"ly interval, so that no matter what interval you specify when evaluating metrics, the step field in window function will always be in terms of days. Look up eval() function using help(c3.ExpressionEngineFunction) for more information.

In [ ]:
# create the failure label metrics
# we use the IsDefective metric for the SmartBulb type to create the failure label metric. 
# do a c3ShowType(ExpressionEngineFunction) on the console to learn more about the window function
failure_metric = c3.CompoundMetric(id = "MyFailureWindow",
                                   name = "MyFailureWindow",
                                   expression = "window('MAX', IsDefective, 0, 120) > 0")
interval = 'DAY'
#While constructing the expressions for the EvalMetricsSpec, we also include HasEverFailed to only get data for lightbulbs that have never failed before
expressions = ['MyFailureWindow',
               'DurationOnInHours',
               'HasEverFailed',
               'StandardDeviationWattsPreviousWeek',
               'SwitchCountPreviousWeek']
#construct the list of features we want to use to build the model
features = ['DurationOnInHours',
            'StandardDeviationWattsPreviousWeek',
            'SwitchCountPreviousWeek']

In [ ]:
# construct the evalMetricsSpec
start = datetime(2014, 1, 1)
end = datetime(2015, 1, 1)
spec = c3.EvalMetricsSpec(expressions = expressions,
                          ids = smb_ids,
                          interval = interval,
                          start = start,
                          end = end)

In [ ]:
# issue the evalMetricsWithMetadata API call to get the data, and tranform to c3.Dataset
res = c3.SmartBulb.evalMetricsWithMetadata(spec=spec,
                                           overrideMetrics=[failure_metric])

dataset = c3.Dataset.fromEvalMetricsResult(result=res)
xy_dataset_source = c3.Dataset(dataset)

In [ ]:
#sanity-check to see if the data has been loaded correctly
c3.Dataset.toPandas(dataset=xy_dataset_source).head(5)

### B. Clean \& Train-Test Split the Dataset<a class="anchor" id="1b"></a>

In [ ]:
# keep only the datapoints where the lightbulb has not failed before 
# remove the column after as we dont need it to build the model
xy_dataset = c3.Dataset.rejectRows(this=xy_dataset_source, conditionColumn='HasEverFailed')
xy_dataset = c3.Dataset.dropColumns(this=xy_dataset, columns=['HasEverFailed'])

In [ ]:
#eyeball the dataset
c3.Dataset.toPandas(dataset=xy_dataset).head(5)

In [ ]:
#listing features
features

In [ ]:
# randomly split the dataset into 30% testing and 70% training
# split is stratified, so both train and test sets have same distribution of MyFailureWindow
xy_test_dataset, xy_train_dataset = c3.Dataset.split(this=xy_dataset,
                                                     fraction=0.3,
                                                     stratificationColumn='MyFailureWindow')
#both xy_train_dataset and xy_test_dataset datasets have both features and target label
xy_test_dataset = c3.Dataset(xy_test_dataset)
xy_train_dataset = c3.Dataset(xy_train_dataset)

In [ ]:
#split the training and test sets into features and labels by extracting the relevant columns using extractColumns
x_train_dataset = c3.Dataset.extractColumns(this=xy_train_dataset,
                                            columns=features)
y_train_dataset = c3.Dataset.extractColumns(this=xy_train_dataset,
                                            columns=['MyFailureWindow'])
x_test_dataset = c3.Dataset.extractColumns(this=xy_test_dataset,
                                           columns=features)
y_test_dataset = c3.Dataset.extractColumns(this=xy_test_dataset,
                                           columns=['MyFailureWindow'])

In [ ]:
x_train_dataset = c3.Dataset(x_train_dataset)
y_train_dataset = c3.Dataset(y_train_dataset)
x_test_dataset = c3.Dataset(x_test_dataset)
y_test_dataset = c3.Dataset(y_test_dataset)

In [ ]:
y_test_dataset.shape

### C. Normalize the Features<a class="anchor" id="1c"></a>

In [ ]:
# construct a SklearnPipe that does standard scaling on the data
scaler = c3.SklearnPipe(name='standardScaler',
                        noTrainScore=False,
                        technique=c3.SklearnTechnique(name='preprocessing.StandardScaler',
                                                      processingFunctionName='transform',
                                                      keepInputColumnIndices=True))

In [ ]:
# train the standard scaler on training data
scalerTrained = c3.SklearnPipe.train(this=scaler, input=x_train_dataset)

In [ ]:
# scale both training and testing using the trained standard scaler
# Make sure not to fit the scaler on the entire dataset(train + test)
x_train_dataset_normalized = c3.SklearnPipe.process(this=scalerTrained,
                                                     input=x_train_dataset)
x_test_dataset_normalized = c3.SklearnPipe.process(this=scalerTrained,
                                                    input=x_test_dataset)

### D. Eyeball the Data<a class="anchor" id="1d"></a>

In [ ]:
c3.Dataset.toPandas(dataset=c3.Dataset(x_train_dataset)).head()

In [ ]:
c3.Dataset.toPandas(dataset=c3.Dataset(x_train_dataset_normalized)).head()

In [ ]:
c3.Dataset.toPandas(dataset=c3.Dataset(y_train_dataset)).head()

In [ ]:
c3.Dataset.toPandas(dataset=c3.Dataset(y_train_dataset)).MyFailureWindow.value_counts()

In [ ]:
c3.Dataset.toPandas(dataset=c3.Dataset(y_test_dataset)).MyFailureWindow.value_counts()

We can see that both train and test datasets have the same distribution of failure labels

# Section 2- Construct a Deep Neural Network using Tensorflow's low-level API<a class="anchor" id="2"></a>

In Section 2, we will construct a deep neural network using Tensorflow's low-level API, and run both forward propogation and backward propogation. The goal is to get familiar with two basic concepts in the world of tensorflow, the tensor and the computational graph.

<img src="http://learnc3.ai/wp-content/uploads/2020/01/TensorflowCoreWalkThrough.png" width="700">

For more information on how to use Tensorflow, visit the [official tensorflow tutorials website](https://www.tensorflow.org/tutorials).

The following graph visualizes the deep neural network that we are going to build. Each input sample has **three features**. There are **two hidden layers**, each consists of four neurons. The **output layer is the probability** of failure. The **hidden layers** can have more neurons (we will experiment with 16). You can definitely try fewer and more. Can you think of how this might have an effect on training and computational resources

<img src="http://learnc3.ai/wp-content/uploads/2020/01/ComputationalGraph.png" width="700">

### A. Construct the Computational Graph<a class="anchor" id="2a"></a>

In [ ]:
# Model Parameters
n_hidden_1 = 16  # the number of neurons in the 1st hidden layer
n_hidden_2 = 16  # the number of neurons in the 2nd hidden layer
num_features = 3 # the size of the input data, or the number of features in one data point
num_classes = 1 # this is a binary classifier

# Tensorflow Graph Input
x = tf.placeholder(dtype=tf.float32, shape=[None, num_features])
y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

# Tensorflow DNN parameters, created using model parameters defined above
weights = {
    "w1": tf.Variable(tf.random_normal(shape=[num_features, n_hidden_1])),
    "w2": tf.Variable(tf.random_normal(shape=[n_hidden_1, n_hidden_2])),
    "w3": tf.Variable(tf.random_normal(shape=[n_hidden_2, num_classes]))
}

bias = {
    "b1": tf.Variable(tf.random_normal(shape=[n_hidden_1])),
    "b2": tf.Variable(tf.random_normal(shape=[n_hidden_2])),
    "b3": tf.Variable(tf.random_normal(shape=[num_classes]))
}

# Construct the computational graph
def dnn(x):
    # hidden layer 1: fully connected with 4 neurons; 
    #                 activation function is Sigmoid.
    z1 = tf.add(tf.matmul(x, weights['w1']), bias['b1'])
    a1 = tf.nn.sigmoid(z1, name='a1')
    
    # hidden layer 2: fully connected with 4 neurons;
    #                 activation function is Sigmoid.
    z2 = tf.add(tf.matmul(a1, weights['w2']), bias['b2'])
    a2 = tf.nn.sigmoid(z2, name='a2')
    
    # output layer: fully connected with 1 neuron;
    #               activation function is Sigmoid.
    z3 = tf.add(tf.matmul(a2, weights['w3']), bias['b3'])
    a3 = tf.nn.sigmoid(z3, name='a3')
    
    return a3

For more information on how to create a tf graph, placeholders, and tf sessions visit [this link](https://medium.com/@d3lm/understand-tensorflow-by-mimicking-its-api-from-scratch-faa55787170d) 

### B. Run Forward Propagation<a class="anchor" id="2b"></a>
Pay close attention to the code in the next cell as you will have to write the code for Forward Propagation of your model as an upcoming exercise.

In [ ]:
with tf.Session() as sess:
    # Run the global variables initializer (calls the initializer for each variable defined)
    sess.run(tf.global_variables_initializer())
    
    # Get the input data
    train_x = c3.Dataset.toPandas(dataset = x_train_dataset_normalized)[features]
    
    # Feed the input data into the computational graph and evaluate
    print(x)
    y_pred = dnn(x)
    ybar_ = sess.run(y_pred, {x: train_x})
    print(ybar_)

### C. Run Backward Propagation (Optional)<a class="anchor" id="2c"></a>

<img src="http://learnc3.ai/wp-content/uploads/2020/01/BackwardPropogation.png" width="700">

In [ ]:
# Construct the DNN model
learning_rate = 0.01
logits = dnn(x=x)

# Define loss and optimizer
loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits, labels=y))
# you could try different optimization techniques for backpropagation, we use Adam here.
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# Evaluate model
correct_pred = tf.equal(tf.round(logits), y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Number of epochs. 
#Can you think of how to best choose the number of epochs for training? Is there more than 1 solution?
num_epochs = 10 
# Number of batches per epoch. How will increasing or reducing the batch size affect the learning curve and training speed?
batch_size = 64
#one epoch is one pass over the entire dataset. So the number of steps per epoch should be length(x_train_dataset)/batch_size
num_steps_perepochs = int(np.floor(c3.Dataset.toPandas(dataset=x_train_dataset).shape[0]/batch_size))

In [ ]:
# Run backward propagation (training)

with tf.Session() as sess:
    # run the global variables initializer
    sess.run(tf.global_variables_initializer())
    #load the dataset for training and testing
    train_x = c3.Dataset.toPandas(dataset=x_train_dataset_normalized)
    train_y = c3.Dataset.toPandas(dataset=y_train_dataset)
    
    for i in range(num_epochs):
        for j in range(num_steps_perepochs):
            #index only the relevant data using the epoch step index and batch size
            this_train_x = train_x.iloc[j*batch_size: (j+1)*batch_size,:]
            this_train_y = train_y.iloc[j*batch_size: (j+1)*batch_size,:]
            #run the session
            sess.run(train_op, 
                     feed_dict={x: this_train_x, y: this_train_y})
            #get loss and accuracy for the batch
            this_loss, this_acc = sess.run([loss, accuracy], 
                                           feed_dict={x: this_train_x, y: this_train_y})
        print("Epoch {}, Loss={}, Training Accuracy={}".format(i+1, this_loss, this_acc))
    print("Backward Propagation (Training Loss Optimization) Finished!")
    test_x = c3.Dataset.toPandas(dataset=x_test_dataset_normalized)
    test_y = c3.Dataset.toPandas(dataset=y_test_dataset)
    test_loss, test_acc = sess.run([loss, accuracy],
                                   feed_dict={x: test_x, y: test_y})
    print("Testing Loss={}, Accuracy={}".format(test_loss, test_acc))

### D. Assignment: Construct Your Own DNN \& Run Forward Propagation<a class="anchor" id="2d"></a>
Construct a DNN with 3 hidden layer, each consists of 5 neurons, and has Rectified Linear Unit (**ReLU**) activation function. We have completed hidden layer 1 for your reference.
>Hint: Use `tf.nn.relu`.

In [ ]:
# Parameters
n_hidden_1 = 5 # the number of neurons in the 1st hidden layer
n_hidden_2 = 5 # the number of neurons in the 2nd hidden layer
n_hidden_3 = 5 # the number of neurons in the 3rd hidden layer
num_features = 3 # the size of the input data, or the number of features in one data point
num_classes = 1 # this is a binary classifier

# Tensorflow Graph Input
x = tf.placeholder(dtype=tf.float32, shape=[None, num_features])
y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

# Tensorflow DNN parameters
weights = {
    "w1": tf.Variable(tf.random_normal(shape=[num_features, n_hidden_1])),
    "w2": tf.Variable(tf.random_normal(shape=[n_hidden_1, n_hidden_2])),
    "w3": tf.Variable(tf.random_normal(shape=[n_hidden_2, n_hidden_3])),
    "w4": tf.Variable(tf.random_normal(shape=[n_hidden_3, num_classes]))
}

bias = {
    "b1": tf.Variable(tf.random_normal(shape=[n_hidden_1])),
    "b2": tf.Variable(tf.random_normal(shape=[n_hidden_2])),
    "b3": tf.Variable(tf.random_normal(shape=[n_hidden_3])),
    "b4": tf.Variable(tf.random_normal(shape=[num_classes]))
}

# Construct the computational graph
# _____FINISH THE FUNCTION_____:
def dnn(x):
    # hidden layer 1: fully connected with 5 neurons; 
    #                 activation function is ReLU.
    z1 = tf.add(tf.matmul(x, weights['w1']), bias['b1'])
    a1 = tf.nn.relu(z1, name='a1')

    
    # hidden layer 2: fully connected with 5 neurons;
    #                 activation function is ReLU.
    # ____YOUR CODE GOES HERE____:    

    
    # hidden layer 3: fully connected with 5 neurons;
    #                 activation function is ReLU.
    # ____YOUR CODE GOES HERE____:    

    
    # output layer: fully connected with 1 neuron;
    #               activation function is ReLU.
    # ____YOUR CODE GOES HERE____:
  
    
    return 

In [ ]:
# Run forward propagation:
# ____FINISH THE SESSION RUN CODE____:
with tf.Session() as sess:
    # Run the initializer
    sess.run(tf.global_variables_initializer())
    # Get the input data
    train_x = c3.Dataset.toPandas(dataset=
        x_train_dataset_normalized)[features]
    
    # Feed the input data into the computational graph and evaluate
    # ___YOUR CODE GOES HERE____:
    
    
    #  code cHint: print(ybar_)

# Section 3 - Train the Deep Neural Network (DNN) on the Platform<a class="anchor" id="4"></a>

Now you have seen how to construct and train a Deep Neural Network using the low level api in tensorflow. We created placeholders, variables and an entire computation graph for forward and backward propagation. While the low level api gives us more control, it is very easy to replicate this architecture using Tensorflow's high level api. We achieve the same goal, but with minimal code. The behaviour of the high level tensorflow api estimators is analogous to estimators in sklearn. 

In this section, we are going to construct the Deep Neural Network in Tensorflow's high-level API and train the DNN on our platform. We will also see how to score the DNN and do predictions using the same on our platform.

We have learned the basic archetecture of **`c3.MLPipeline`** in the machine learning module. We also practiced to use some of the **`c3.MLPipeline`**. We will be using **`c3.TensorflowClassifier`** which is **`c3.MLLeafPipe`** to train the DNN on the platform.

You will go through the following steps:

- Construct a DNN classifier using Tensorflow's high-level API and serialize the model object.

    
- Construct a **`c3.TensorflowClassifier`** using the serialized model object. Train the it on the platform. Score the trained classifier, also make predictions using the trained classifier.

    
- Upsert the trained **`c3.TensorflowClassifier`** to the platform, fetched it, and do scoring and predictions using the fetched trained classifier.

    
- Clean up on the platform.

<img src="http://learnc3.ai/wp-content/uploads/2020/01/MLLeafPipe.png" width="700">

### A. Construct a DNN Classifier using Tensorflow's High-Level API<a class="anchor" id="3a"></a>

For more information about using TensorFlow's DNN classifier, visit [this link](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier)

In [ ]:
features_tf = []
#each feature must be converted to a tf.feature_column object (numeric_column in this case)
#feature_column in tensorflow acts as the bridge between raw data and the estimator, by converting raw data into the right input format for the estimator
for feature in features:
    features_tf.append(tf.feature_column.numeric_column(key=feature))
    
#initialize the classifier and serialize the same using the helper function
tf_classifier = tf.estimator.DNNClassifier(feature_columns=features_tf,
                                           hidden_units=[10,10],
                                           activation_fn=tf.nn.relu,
                                           n_classes=2)

tf_classifier_serialized = serialize_compress_python_object(tf_classifier)

tf_classifier_serialized

In [ ]:
tfClassifier = c3.TensorflowClassifier(inputFormat="feature-map",
                                       name="tfClassifier",
                                       predictProbability=False,
                                       scoringMetrics=c3.MLScoringMetric.toScoringMetricMap(scoringMetricList=[c3.MLAccuracyMetric(), 
                                                       c3.MLPrecisionMetric(), 
                                                       c3.MLRecallMetric()]), #list the desired scoring metrics
                                       technique=c3.DeepLearningTechnique(
                                           name='tf_classifier',
                                           modelDef=tf_classifier_serialized, #pass the serialized tensorflow model
                                           hyperParameters={"numSteps": 1000, #pass hyperparameters for training
                                                            "numEpochs": 10, 
                                                            "batchSize": 64}))

### B. Train on the Platform<a class="anchor" id="3b"></a>

In [ ]:
tfClassifier.toJson()

In [ ]:
#train the classifier
try:
    tfClassifierTrained = c3.TensorflowClassifier.train(this=tfClassifier, 
                                                     input=x_train_dataset_normalized,
                                                     targetOutput=y_train_dataset)
except Exception as e:
    print(str(e)[:10000])

In [ ]:
#obtain the training statistics/summary
tfClassifierTrained.trainingStats

In [ ]:
#calculate the training and test scores on the dataset by using c3.TensorflowClassifier.score
#Pass the trained model, input dataset with features to score on and the ground truth
tfClassifierTrained.predictProbability=False
print(c3.TensorflowClassifier.score(this=tfClassifierTrained,
                                    input=x_train_dataset_normalized,
                                    targetOutput=c3.Tensor(y_train_dataset)))
print(c3.TensorflowClassifier.score(this=tfClassifierTrained,
                                    input=x_test_dataset_normalized,
                                    targetOutput=c3.Tensor(y_test_dataset)))

In [ ]:
# use c3.TensorflowClassifier.process to get thr predictions. We need this to plot the ROC and precision-recall curves
tfClassifierTrained.predictProbability = True
yhat_train = c3.TensorflowClassifier.process(this=tfClassifierTrained,
                                                 input=x_train_dataset_normalized)
yhat_test = c3.TensorflowClassifier.process(this=tfClassifierTrained,
                                            input=x_test_dataset_normalized)

In [ ]:
yhat_train_df = c3.Dataset.toPandas(dataset=yhat_train)['class_1']
yhat_test_df = c3.Dataset.toPandas(dataset=yhat_test)['class_1']
y_train_df = c3.Dataset.toPandas(dataset=y_train_dataset)['MyFailureWindow']
y_test_df = c3.Dataset.toPandas(dataset=y_test_dataset)['MyFailureWindow']

In [ ]:
test_auc = plot_roc(y_pred_test=yhat_test_df,
                    y_true_test=y_test_df,
                    y_pred_train=yhat_train_df,
                    y_true_train=y_train_df)
test_avg_prec = plot_prec(y_pred_test=yhat_test_df,
                          y_true_test=y_test_df,
                          y_pred_train=yhat_train_df,
                          y_true_train=y_train_df)

### C. Upsert \& Fetch the Model<a class="anchor" id="3c"></a>

In [ ]:
#upsert the model
tfClassifierTrainedUpserted = c3.TensorflowClassifier.upsert(this=tfClassifierTrained)
tfClassifierTrainedUpserted

In [ ]:
#fetch the upserted model
tfClassifierFetched = c3.TensorflowClassifier.get(this=tfClassifierTrainedUpserted)

In [ ]:
#sanity-check
tfClassifierFetched.toJson()

In [ ]:
#using the fetched model to plot ROC and precition-recall curves
tfClassifierFetched.predictProbability=True
yhat_train = c3.TensorflowClassifier.process(this=tfClassifierFetched,
                                             input=x_train_dataset_normalized)
yhat_test = c3.TensorflowClassifier.process(this=tfClassifierFetched,
                                            input=x_test_dataset_normalized)
yhat_train_df = c3.Dataset.toPandas(dataset=yhat_train)['class_1']
yhat_test_df = c3.Dataset.toPandas(dataset=yhat_test)['class_1']

In [ ]:
test_auc = plot_roc(y_pred_test=yhat_test_df,
                           y_true_test=y_test_df,
                           y_pred_train=yhat_train_df,
                           y_true_train=y_train_df)
test_avg_prec = plot_prec(y_pred_test=yhat_test_df,
                                 y_true_test=y_test_df,
                                 y_pred_train=yhat_train_df,
                                 y_true_train=y_train_df)

### D. Cleanup<a class="anchor" id="3d"></a>

In [ ]:
c3.TensorflowClassifier.fetch()

In [ ]:
#removing the upserted model
c3.TensorflowClassifier.removeAll()

In [ ]:
#sanity-check
c3.TensorflowClassifier.fetch()

### E. Assignment: Construct your own C3 TensorflowClassifer and Train on the Platform<a class="anchor" id="3e"></a>

In the assignment, you need to construct your own deep neural network. You deep neural network has 3 hidden layer, and each layer consists of 5 neurons. We continue to use relu as the activation function.

After you construct your own deep neural network, make a **`c3.TensorflowClassifier`** instance, and train it on the platform. Make sure you have all 3 scoring metrics: accuracy, precision and recall in your C3 leaf pipe.

Notice that you need to use the normalized data when you train the leaf pipe.

You should also be able to check the training statistics by examining the trained leaf pipe. Please refer the example code above (in this section).

In [ ]:
# STEP1: GET THE SERIALIZED TENSORFLOW CLASSIFIER
# ____YOUR CODE GOES HERE____:
tf_classifier_serialized = ???

# STEP2: CONSTRUCT A c3.TensorflowClassifier
# ____YOUR CODE GOES HERE____:
tfClassifier = c3.TensorflowClassifier(???)

# STEP3: TRAIN YOUR c3.TensorflowClassifier ON THE PLATFORM
# ____YOUR CODE GOES HERE____:
tfClassifierTrained = c3.TensorflowClassifier.train(???)

# STEP4: LOOK AT THE TRAINING PERFORMANCE
# tfClassifierTrained.trainingStats

# Section 4 - Train the ML Pipeline with DNN on the Platform<a class="anchor" id="4"></a>

In this section, we will construct a serial pipeline that use both sklearn and tensorflow techniques. The type system and the machine learning pipeline built on the C3 AI platform makes composing techniques from both sklearn and tensorflow very easy and intuitive.

In the previous section, we trained the Tensorflow classifier using the normalized data set. In this section, however, we will compose both the normalization step and the classification step in **one coherent machine learning pipeline**. The pipeline can then be trained, scored, and used to make predictions **on the platform**.

We will also be reviewing the archetecture of `c3.MLPipeline`, `c3.MLLeafPipe`, and `c3.MLSerialPipe`.

In this section, you will go through the following four steps:

**Step 1**: Construct a DNN classifier using Tensorflow's high-level API and serialize the model object;

**Step 2**: Construct a `c3.MLSerialPipeline` using the serialized model object. Train it on the platform and score the trained pipeline. Here, you'll also make predictions using the trained pipeline.

**Step 3**: Upsert the trained `c3.MLSerialPipeline` to the platform, fetch it, and do scoring and predictions using the fetched trained pipeline.
    
**Step 4**: Clean up on the platform.

<img src="http://learnc3.ai/wp-content/uploads/2020/01/MLPipe-7.8.8.png" width="700">

### A. Construct an ML Pipeline that Consists of a DNN Classifier<a class="anchor" id="4a"></a>

In [ ]:
#each feature must be converted to a tf.feature_column object (numeric_column in this case)
#feature_column in tensorflow acts as the bridge between raw data and the estimator, by converting raw data into the right input format for the estimator
features_tf = []
for feature in features:
    features_tf.append(tf.feature_column.numeric_column(key=feature))
#initialize the classifier and serialize the same using the helper function
tf_classifier = tf.estimator.DNNClassifier(feature_columns=features_tf,
                                           hidden_units=[10,10],
                                           activation_fn=tf.nn.relu,
                                           n_classes=2)
tf_classifier_serialized = serialize_compress_python_object(tf_classifier)

<img src="http://learnc3.ai/wp-content/uploads/2020/01/MLSerialDetails.png" width="700">

In [ ]:
# set up the scaler in an sklearn pipe
scaler = c3.SklearnPipe(name='standardScaler',
                        noTrainScore=False,
                        technique=c3.SklearnTechnique(name='preprocessing.StandardScaler',
                                                      processingFunctionName='transform',
                                                      keepInputColumnIndices=True))

tfClassifier = c3.TensorflowClassifier(inputFormat="feature-map",
                                       name="tfClassifier",
                                       predictProbability=False,
                                       technique=c3.DeepLearningTechnique(
                                           name='tf_classifier',
                                           modelDef=tf_classifier_serialized,
                                           hyperParameters={"numSteps": 1000, 
                                                            "numEpochs": 10, 
                                                            "batchSize": 64}))
# Construct an MLSerialPipeline with both scaling and classification steps along with scoring metrics
mlcl = c3.MLSerialPipeline(name='mlClassifier',
                                       steps=[c3.MLStep(name='standardScalerStep',
                                                                    pipe=scaler),
                                              c3.MLStep(name='tfClassifierStep',
                                                                    pipe=tfClassifier)],
                                              scoringMetrics=c3.MLScoringMetric.toScoringMetricMap(scoringMetricList=[c3.MLAccuracyMetric(), 
                                                       c3.MLPrecisionMetric(), 
                                                       c3.MLRecallMetric()]))

### B. Train on the Platform<a class="anchor" id="4b"></a>

In [ ]:
try:
    mlclTrained = c3.MLSerialPipeline.train(this=mlcl,
                                            input=x_train_dataset,
                                            targetOutput=y_train_dataset)
except Exception as e:
    print(str(e)[:1000])

In [ ]:
#obtain training statistics
mlclTrained.steps[1].pipe.trainingStats

In [ ]:
# extract the parameters of the scaler
mlclTrained.steps[0].pipe.trainedModel.parameters

In [ ]:
# extract the parameters of the model
mlclTrained.steps[1].pipe.trainedModel.parameters

In [ ]:
# obtain the train and test scores
mlclTrained.steps[1].pipe.predictProbability=False
c3.MLSerialPipeline.score(this=mlclTrained,
                                      input=x_train_dataset,
                                      targetOutput=y_train_dataset)
c3.MLSerialPipeline.score(this=mlclTrained,
                                      input=x_test_dataset,
                                      targetOutput=y_test_dataset)

In [ ]:
# use c3.MLSerialPipeline.process method to get the predictions. We need this to plot the ROC and precision-recall curves
mlclTrained.steps[1].pipe.predictProbability=True
yhat_train = c3.MLSerialPipeline.process(this=mlclTrained,
                                                     input=x_train_dataset)
yhat_test = c3.MLSerialPipeline.process(this=mlclTrained,
                                                    input=x_test_dataset)
yhat_train_df = c3.Dataset.toPandas(dataset=yhat_train)['class_1']
yhat_test_df = c3.Dataset.toPandas(dataset=yhat_test)['class_1']

In [ ]:
y_train_df = c3.Dataset.toPandas(dataset=y_train_dataset)['MyFailureWindow']
y_test_df = c3.Dataset.toPandas(dataset=y_test_dataset)['MyFailureWindow']

In [ ]:
test_auc = plot_roc(y_pred_test=yhat_test_df,
                           y_true_test=y_test_df,
                           y_pred_train=yhat_train_df,
                           y_true_train=y_train_df)
test_avg_prec = plot_prec(y_pred_test=yhat_test_df,
                                 y_true_test=y_test_df,
                                 y_pred_train=yhat_train_df,
                                 y_true_train=y_train_df)

### C. Upsert \& Fetch<a class="anchor" id="4c"></a>

In [ ]:
mlclTrainedUpserted = c3.MLSerialPipeline.upsert(this=mlclTrained)
mlclTrainedUpserted

In [ ]:
mlclFetched = c3.MLSerialPipeline.get(this=mlclTrainedUpserted)

> **Note**: If you're running into memory issues, run the cell below. If not, no need to run it!

In [ ]:
#del mlclFetched._data['steps'][1]['pipe']['trainedModel']['modelFile']['type']

In [ ]:
mlclFetched.steps[1].pipe.predictProbability=True
yhat_train = c3.MLSerialPipeline.process(this=mlclFetched,
                                                     input=x_train_dataset)
yhat_test = c3.MLSerialPipeline.process(this=mlclFetched,
                                                    input=x_test_dataset)
yhat_train_df = c3.Dataset.toPandas(dataset=yhat_train)['class_1']
yhat_test_df = c3.Dataset.toPandas(dataset=yhat_test)['class_1']

In [ ]:
test_auc = plot_roc(y_pred_test=yhat_test_df,
                           y_true_test=y_test_df,
                           y_pred_train=yhat_train_df,
                           y_true_train=y_train_df)
test_avg_prec = plot_prec(y_pred_test=yhat_test_df,
                                 y_true_test=y_test_df,
                                 y_pred_train=yhat_train_df,
                                 y_true_train=y_train_df)

### D. Cleanup <a class="anchor" id="4d"></a>

In [ ]:
c3.MLSerialPipeline.fetch()

In [ ]:
c3.MLSerialPipeline.removeAll()

In [ ]:
c3.MLSerialPipeline.fetch()

### E. Assignment: Construct your own C3 ML Pipeline with DNN and train on the platform<a class="anchor" id="4e"></a>

In the assignment, you need to construct your own deep neural network. You deep neural network has 3 hidden layer, and each layer consists of 5 neurons. We continue use relu as the activation function.

After you construct your own deep neural network, make a **`c3.MLSerialPipeline`** instance. The serial pipeline has two step, the first step is normalizing the features using the standard scaler from scikit learn, and the second step is doing classification using the DNN. Once you composed the serial pipeline, train it on the platform. Make sure you have all 3 scoring metrics: accuracy, precision and recall in your serial pipeline.

Notice that you do not need to use the normalized data when you train the leaf pipe. 

You should also be able to check the training statistics by examining the dnn step in the serial pipeline. 

In [ ]:
# STEP1: GET THE SERIALIZED TENSORFLOW CLASSIFIER
# ____YOUR CODE GOES HERE____:
tf_classifier_serialized = ???


# STEP2: CONSTRUCT A c3.MLTensorTensorSerialPipeline
# ____YOUR CODE GOES HERE____:
# step one: normalize features using sklearn - standardscaler
scaler = ???
# step two: build a DNN classifier
tfClassifier = ???
# step three: compose the serial pipeline
mlcl = c3.MLSerialPipeline(name='mlClassifier',
                           steps=???,
                           scoringMetrics=???)


# STEP3: TRAIN YOUR c3.MLTensorTensorSerialPipeline ON THE PLATFORM
# ____YOUR CODE GOES HERE____:
try:
    mlclTrained = c3.MLSerialPipeline.train(???)
except Exception as e:
    print(str(e)[:1000])


# STEP4: EXAMINE THE TRAINING STATS

# mlclTrained.steps[1].pipe.trainingStats

## Conclusion

Congratulations! You have successfully traversed the deep learning landscape of the C3 AI Suite!!

## Appendix - Kernel Installation Instructions <a class="anchor" id="1.4">

If the **`py-deepLearning`** kernel is not available, perform the following:
1. Select the 'Kernel' tab at the top of this notebook, then select 'Manage Kernels';
2. Scroll to find `py-deepLearning`, and click the 'Install' button;
> Note: This installation will "fail", but don't worry, exit the Manage Kernels UI and return to the main notebook.
3. Navigate to the 'File' tab located at the top of the notebook, and select 'Close and Hault'. This will close the notebook. Re-open it.
4. You should notice, in the upper-right corner, the kernel is now set correctly to `py-deepLearning`. If it is not, navigate to the Kernel tab, then select 'Change Kernel' and select the appropriate option.
    
## Make sure to CLOSE AND HALT  (from bottom of the File menu at the top) this notebook when you leave.